## Environment Setup
**Make sure to set your runtime to use a GPU by going to `Runtime` -> `Change runtime type` -> `Hardware accelerator` -> `T4 GPU`**

In [ ]:
import sys
if 'google.colab' in sys.modules:  # If in Google Colab environment
    # Mount google drive to enable access to data files
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Installing requisite packages
    !pip install transformers accelerate pymupdf &> /dev/null
    
    # Change working directory to health
    %cd /content/drive/MyDrive/LLM4BeSci_GSERM2024/day_5

In [2]:
import fitz
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

## Load Data and Model

In [5]:
# Extract text from PDF
pdf = fitz.open('riskEmbeddings.pdf')
text = ""
for page in pdf:
    text += page.get_text()

text[:1000]  # Display the first 500 characters to verify content extraction

'Hussain et al. EPJ Data Science           ( 2024)  13:38 \nhttps://doi.org/10.1140/epjds/s13688-024-00478-x\nR ES EA RCH\nOpen Access\nNovel embeddings improve the prediction\nof risk perception\nZak Hussain1,2*\n, Rui Mata1 and Dirk U. Wulﬀ1,2\n*Correspondence:\nz.hussain@unibas.ch\n1Faculty of Psychology, University of\nBasel, Missionsstrasse 60–62, Basel,\n4055, Switzerland\n2Center for Adaptive Rationality,\nMax Planck Institute for Human\nDevelopment, Lentzeallee 94, Berlin,\n14195, Germany\nAbstract\nWe assess whether the classic psychometric paradigm of risk perception can be\nimproved or supplanted by novel approaches relying on language embeddings. To\nthis end, we introduce the Basel Risk Norms, a large data set covering 1004 distinct\nsources of risk (e.g., vaccination, nuclear energy, artiﬁcial intelligence) and compare\nthe psychometric paradigm against novel text and free-association embeddings in\npredicting risk perception. We ﬁnd that an ensemble model combining text 

In [ ]:
torch.random.manual_seed(42) # For reproducibility

# Load the model 
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct",
    device_map="cuda", # Use GPU
    torch_dtype=torch.float16, # Use half-precision
    trust_remote_code=True, 
    attn_implementation='eager' # For faster inference on T4 GPUs
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 1000,  # Maximum number of tokens to generate
    "return_full_text": False, # Return only the generated text
    "do_sample": False # Use greedy decoding
}

In [ ]:
def query_paper(query):
    prompt = [{"role": "user", "content": query + '\n-------------------\n' + text}]
    output = pipe(prompt, **generation_args)
    return output[0]['generated_text']

q_1 = 

In [ ]:
q_1 = "What is the title of the following paper?\n-------------------------\n"
print(pipe([{"role": "user", "content": q_1 + text}])[0]['generated_text'])

In [ ]:
q_2 = "Return to me the full abstract of the following paper:\n-------------------------\n"
print(pipe([{"role": "user", "content": q_2 + text}])[0]['generated_text'])

In [ ]:
q_3 = "Summarise the following paper:\n-------------------------\n"
print(pipe([{"role": "user", "content": q_3 + text}])[0]['generated_text'])